Self Attention付きのLSTMを用いて、文章を分類したり予測するモデルを作る。  

感情分析はよくやられているので、今回はSelf attention付きのLSTMを用いた文章の分類モデルを作ります。  
他クラス分類は未知のクラスに脆弱だったり、複数クラスに属する可能性(タグ付けなど)を考慮出来ません。  
そこで、今回はSelf attentionを用いて文章があるクラスに属するかどうかを調べる2値分類モデルを作成します。

# ゴール

未知のレビューに対して、6クラスそれぞれに属するかどうかを判定するモデルを作成する 

# 確認すべき事

各クラスに対して、以下を確認する  

2値分類のモデルの作成→回帰[0, 1]で解くか、分類[0, 1] or [1, 0]で解くか  
↓  
結局やってることは変わらない、Binary Cross Entropyを用いて解く、出力はスカラー(1になる確率)  
他クラス分類であれば$[p_1, p_2, p_3]$とベクトルを渡すが、2クラスなら$[p_1, p_2]$を渡すか、$p$を渡すかの違い  
$p$を渡せば、自動的に$p_2$が求まる  
ただ、pytorchの関数上では使用する関数が異なるので注意、torch.nnなのかtorch.nn.functionalなのか  
nnとfunctionalの違いはしんどいので深追いしない  

precision, recallの確認  
評価手法はまだ未確定、要検討  
通常のLSTMと、self-attention付きLSTMで性能比較→全部やらなくても、単一カテゴリでいいか  
どの単語にAttentionしているのかを可視化  

未知のレビューに対する分類
他クラス分類(1of n)にしたい場合は、所属する確率が最大になるモデルを採択  
タグ付け(k of n)にしたい場合は、出力の確率が0.5より大きいかで判断する  

# 作業工程

#### データセットの準備  
カテゴリ毎のデータのダウンロード  
trainとtest、その他にtextを切り出し  
カテゴリ毎に保存  
カテゴリ名を指定すると、ダウンサンプリングされたtrain_data, test_dataが返ってくる関数を作成  

#### 全ての語彙の格納  
全カテゴリのデータの語彙を格納したVocabを作成  
Vocabは単語⇄インデックスのやり取りや、語彙数の管理を行う  
オンライン学習(未知の単語への対応)はまだよくわかっていない  
格納したvocabを保存しておく  

#### DataIteratorの作成  
train_data, test_dataを元にインスタンスを作成し、forループを回すと自動的にバッチを作成するclassを定義  
バッチは全ての単語がインデックスに変換されており、バッチ内で一番長い文の長さに合わせて0埋めされている  

#### モデルの作成  
入力: 文章  
出力: スカラー(所属確率)  
となるモデルを作成する  
通常のLSTMと、self attention付きのLSTM2パターンを作成する  

#### 学習に必要な関数の定義と学習の実行  
損失関数や学習関数、パラメータの設定  
同一iteration内で行う処理をまとめた関数の設定  
KFoldでデータを5分割して、1つをvalidationとして使用(つまり、1回のiterationで5回学習を行う)  
ラベルの偏りが出ないように、ラベルの値毎に層別で分割を行う  
同一iterationのvalid_lossは、KFoldした値の平均値を採用
learning_curveの監視の仕組みを整備  
early stopping  
学習  

#### 学習済みモデルの保存  
モデルとモデルのパラメータを保存  

#### モデルの性能評価  
test_dataに対して混同行列とF1-scoreを計算して見せる  
カテゴリ毎の分類で、いい感じの分類器と微妙な分類器を見せる  
上手くいったパターン、上手くいかなかったパターンの文章をself-attention付きで確認する  
self-attentionありのモデルと無しのモデルでF1-scoreがどれだけ変わるのかをまとめる  

#### 結論  
まとめる

notebookを分割した方が良さそう  
このノートブックは、データの整形、前処理を行う  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
import re
from collections import defaultdict
import glob


np.random.seed(1234)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
sns.set_style('darkgrid')

%matplotlib inline

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
import torch
import torch.nn as nn
from torch import optim
from torch.nn import functional as F

In [3]:
torch.manual_seed(1234)

In [4]:
device = torch.device( "cuda" if torch.cuda.is_available() else "cpu")

In [5]:
print(torch.__version__)

0.4.0


Amazon review dataを使います。  
URL: http://jmcauley.ucsd.edu/data/amazon/

これは、Amazonの様々なカテゴリの商品に対するレビューが集まったデータセットです。  
20万レビューを超えているカテゴリを6つ選び、それぞれ10万サンプルをtrain, 残りの10万サンプルをtestに使用します。  
それぞれのカテゴリのtrain_dataを合算し、それぞれのカテゴリに対して、そのカテゴリに属しているのかどうかを求める2値分類モデルを作成します。  

Video Games  
Home and Kitchen  
Apps for Android  
Health and Beauty  
Clothing, Shoes and Jewelry  
Sports and Outdoors  

train_dataは、そのカテゴリに属しているデータが10万件、属していないデータが50万件とアンバランスなデータになっています。  
そこで、ダウンサンプリングを行い、そのカテゴリに属していないデータを10万件に減らします。  

# データセットの準備

カテゴリ毎に、データ数 × 1のDFを作成する  
カテゴリ: aa, csj, hpc, hk, so, vg  
train: 10万行, test: 10万行

In [6]:
import pathlib

In [7]:
p = pathlib.Path(".").glob("../input/reviews_*.json")
for category, path in zip(["vg", "hk", "so", "csj", "hpc", "aa"], p):
    data = pd.read_json(path, lines=True)
    data = data["reviewText"].str.lower().str.replace("[^\sa-z'-]", "").str.replace(r"\s{2,}", r" ")
    data = data[~data.isin(["", " "])].reset_index(drop=True)
    idx = np.arange(data.shape[0])
    np.random.shuffle(idx)
    train_idx = idx[:1000]
    test_idx = idx[1000:2000]
    train_s = data.loc[train_idx]
    test_s = data.loc[test_idx]
    train_s.to_csv('../preprocessed/{}_train.csv'.format(category), index=False, encoding="utf-8")
    test_s.to_csv('../preprocessed/{}_test.csv'.format(category), index=False, encoding="utf-8")
    print(category, path)
    print(train_s.shape, test_s.shape)

vg ../input/reviews_Video_Games_5.json
(1000,) (1000,)
hk ../input/reviews_Home_and_Kitchen_5.json
(1000,) (1000,)
so ../input/reviews_Sports_and_Outdoors_5.json
(1000,) (1000,)
csj ../input/reviews_Clothing_Shoes_and_Jewelry_5.json
(1000,) (1000,)
hpc ../input/reviews_Health_and_Personal_Care_5.json
(1000,) (1000,)
aa ../input/reviews_Apps_for_Android_5.json
(1000,) (1000,)


reviewTextに空白のレビューがあるせいでCSVで吐き出したレビューと吐き出す前で行数が一致していない  
予め空白がある行を削除する  
要らない文字を消す(""にreplaceする)→余分な空白を消す(" "にreplaceする)→["", " "]に一致する行を消去する  
の手順を踏む

# 単語をintに変換するための辞書作成

NNモデルにデータを投入するためには、単語(str)を数字(int)に変換する必要があります。  
そのため、train_data, test_dataの単語をintに変換するためのクラスを作ります。 

Pytorchのチュートリアルが参考になります。  
URL :https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#sphx-glr-intermediate-seq2seq-translation-tutorial-py

In [6]:
class Vocab:
    def __init__(self):
        self.word2index = defaultdict(int)
        self.word2count = defaultdict(int)
        self.index2word = defaultdict(str)
        self.n_words = 0
    def add_sentence(self, sentence):
        for word in sentence.split(" "):
            self.add_word(word)
    
    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 0
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [7]:
vocab = Vocab()

In [8]:
%%time
for path in glob.glob('../preprocessed/*.csv'):
    series = pd.read_csv(path, header=None, dtype={0: str}, encoding='utf-8').dropna(axis=0)[0]
    for sentence in series:
        vocab.add_sentence(sentence)

CPU times: user 1min 7s, sys: 476 ms, total: 1min 7s
Wall time: 1min 7s


In [9]:
# defaultdictは未知のkeyに対応するvalueを要求すると、defaultのvalueを作成してしまう
# 後々のバグを防ぐため、通常のdictに変えてロックする
vocab.word2index = dict(vocab.word2index)
vocab.index2word = dict(vocab.index2word)
vocab.word2count = dict(vocab.word2count)

# ジェネレータの作成

カッコいいので、ループするとバッチを返すジェネレータを作ります。  
ジェネレータはイテレータを作成する関数のことをさします。  
各バッチは、(レビュアー数, 最大文章長さ)となり、文章の長さがバッチ内最大の長さに足りない部分はゼロ埋めされます。  
ジェネレータを以下のようにクラスで定義すると、何度でも使用する事が出来ます。

参考: https://www.lifewithpython.com/2015/11/python-create-iterator-protocol-class.html

バッチを作成する関数

In [10]:
def make_padded_array(reviews, vocab=vocab):  
    review_list = list()
    len_list = list()
    for r in reviews:
        review_indexes = [vocab.word2index[w] for w in r.split()]
        review_list.append(review_indexes)
        len_list.append(len(review_indexes))
    
    len_array = np.sort(len_list)[::-1].copy() # torch.Tensorは配列逆にしているとエラーを起こすので、コピーする
    idxes = np.argsort(len_list)[::-1].copy()
    text_array = np.zeros((len(review_list), max(len_list)), dtype=int)
    for i, idx in enumerate(idxes):
        text_array[i, :len(review_list[idx])] = review_list[idx]
    return text_array, len_array, idxes + reviews.index[0] # idxesは0スタートなので、入力reviewsのindexと一致するように調整する

バッチを適宜作成する関数

In [76]:
class BatchIterator(object):
    def __init__(self, df, batch_len):
        self.df = df
        self.batch_len = batch_len
        self.n_data = df.shape[0]
    
    def __iter__(self):
        df = self.df.sample(frac=1).reset_index(drop=True) # DFをシャッフルする
        for b_idx in range(0, self.df.shape[0], self.batch_len):
            text_batch = df.loc[b_idx:b_idx+self.batch_len-1, "text"]
            target_batch = df.loc[b_idx:b_idx+self.batch_len-1, "label"]
            
            text_array, len_array, idxes = make_padded_array(text_batch)
            # negative stride(降順に並び替え)するとtorch.LongTensor()が使えない、copy()して新しいメモリにarrayを作り変える
            target_array = target_batch[idxes].values # batch内で順番を並び替えているので、targetもそれに合わせる
            
            
            text_tensor = torch.LongTensor(text_array).to(device)
            lengths_tensor = torch.LongTensor(len_array).to(device)
            target_tensor = torch.LongTensor(target_array).to(device)
            
            yield text_tensor, lengths_tensor, target_tensor

In [122]:
text_iterator = BatchIterator(tmp, 10)

In [123]:
cnt = 0
for te, l, ta in text_iterator:
    print(te.shape, l.shape, ta.shape)
    cnt += 1
    if cnt >= 10:
        break

torch.Size([10, 76]) torch.Size([10]) torch.Size([10])
torch.Size([10, 43]) torch.Size([10]) torch.Size([10])
torch.Size([10, 580]) torch.Size([10]) torch.Size([10])
torch.Size([10, 165]) torch.Size([10]) torch.Size([10])
torch.Size([10, 115]) torch.Size([10]) torch.Size([10])
torch.Size([10, 81]) torch.Size([10]) torch.Size([10])
torch.Size([10, 80]) torch.Size([10]) torch.Size([10])
torch.Size([10, 93]) torch.Size([10]) torch.Size([10])
torch.Size([10, 86]) torch.Size([10]) torch.Size([10])
torch.Size([10, 40]) torch.Size([10]) torch.Size([10])
